In [ ]:
import pandas as pd
import numpy as np
import os


import datetime as dt
from sklearn.preprocessing import OneHotEncoder

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

# allow more data columns to be shown than by default
pd.set_option('display.max_columns', 100)

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# DATA

In [ ]:
# transaction data
cwd_path = os.path.abspath(os.getcwd())
project_root = os.path.dirname(cwd_path)
data_path = os.path.join(project_root, 'data/credit_card_transactions-ibm_v2.csv')
transactions = pd.read_csv(data_path)
display(transactions)

In [ ]:
# keep the original data
transactions_original = transactions.copy()

In [ ]:
# take a random subset of the original data with a smaller size, makes initial data exploration faster 
sample_size = 10**6
transactions = transactions_original.sample(sample_size, random_state=42)
display(transactions)

In [ ]:
# card data
data_path = os.path.join(project_root, 'data/sd254_cards.csv')
cards = pd.read_csv(data_path)
# renanme CARD INDEX to match other data sources
cards = cards.rename(columns={'CARD INDEX':'Card'})
display(cards)

In [ ]:
# user data
data_path = os.path.join(project_root, 'data/sd254_users.csv')
users = pd.read_csv(data_path)
# add User as a column
users['User'] = users.index
display(users)

# MERGE DATA

In [ ]:
# merge transactions and card data
data = transactions.merge(cards, how='inner', on=['User', 'Card'])
display(data)

In [ ]:
# merge user data
data = data.merge(users, how='inner', on='User')
display(data)

# DATA WRANGLING

In [ ]:
for col in data.columns:
    n_unique = data[col].nunique()
    print(f'Number of unique values in {col}: {n_unique}')
    if n_unique<=10:
        print(f'The unique values in column {col}')
        print(data[col].unique())

In [ ]:
data["Is Fraud?"] = [1 if x == "Yes" else 0 for x in data["Is Fraud?"]]
data['Is Fraud?'].sum() / data.shape[0]

In [ ]:
# ONE HOT ENCODING for the different transaction types
one_hot_encoder = OneHotEncoder(sparse=False).fit(np.array(data['Use Chip']).reshape(-1,1))
new_columns = [col[3:] for col in one_hot_encoder.get_feature_names()]
data[new_columns] = one_hot_encoder.transform(np.array(data['Use Chip']).reshape(-1,1))
data

In [ ]:
# Add DATETIME (no timezone info given)
data["Datetime"] = data.apply(lambda row: 
                          dt.datetime(row["Year"], row["Month"], row["Day"], int(row["Time"][0:2]), int(row["Time"][3:5])), 
                          axis=1)
# sort data by datetime
data = data.sort_values(by='Datetime')
# reindex dataframe
data = (data
        .reset_index()
        .rename(columns={'index':'transaction_index'})
       )
display(data)

In [ ]:
# Change transaction AMOUNT from string to float
data['Amount'] = data['Amount'].str.replace('$','').astype(float)
data

In [ ]:
# 0-1 encoding of "Is Fraud?" 
fraud_encoding = {'No':0, 'Yes':1}
# data['Is Fraud?'] = data['Is Fraud?'].map(fraud_encoding)
display(data['Is Fraud?'].unique())
display(data)

In [ ]:
# 0-1 encoding of "Has Chip" 
has_chip_encoding = {'NO':0, 'YES':1}
data['Has Chip'] = data['Has Chip'].map(has_chip_encoding)
unique_values = data['Has Chip'].unique()
print(f'Has Chip unique values: {unique_values}')
display(data)

In [ ]:
# 0-1 encoding of "Gender" 
gender_encoding = {'Male':0, 'Female':1}
data['Gender'] = data['Gender'].map(gender_encoding)
unique_values = data['Gender'].unique()
print(f'Gender unique values: {unique_values}')
display(data)

In [ ]:
print('Card Brand value counts:')
display(data['Card Brand'].value_counts())
print('Card Brand frequencies:')
display(data['Card Brand'].value_counts() / data.shape[0])

In [ ]:
print('Proportion of fraud by Card Brand:')
display(data.loc[data['Is Fraud?']==1, 'Card Brand'].value_counts() / data['Card Brand'].value_counts())

In [ ]:
# ONE HOT ENCODING for the different card brands
one_hot_encoder = OneHotEncoder(sparse=False).fit(np.array(data['Card Brand']).reshape(-1,1))
new_columns = [col[3:] for col in one_hot_encoder.get_feature_names()]
data[new_columns] = one_hot_encoder.transform(np.array(data['Card Brand']).reshape(-1,1))
data

In [ ]:
print('Card Type frequencies:')
display(data['Card Type'].value_counts() / data.shape[0])

print('Proportion of fraud by Card Type:')
display(data.loc[data['Is Fraud?']==1, 'Card Type'].value_counts() / data['Card Type'].value_counts())

In [ ]:
# ONE HOT ENCODING for the different card types
one_hot_encoder = OneHotEncoder(sparse=False).fit(np.array(data['Card Type']).reshape(-1,1))
new_columns = [col[3:] for col in one_hot_encoder.get_feature_names()]
data[new_columns] = one_hot_encoder.transform(np.array(data['Card Type']).reshape(-1,1))
data

In [ ]:
# Change CREDIT LIMIT from string to float
data['Credit Limit'] = data['Credit Limit'].str.replace('$','').astype(float)
data

In [ ]:
# 0-1 encoding of "Card on Dark Web" 
dark_web_encoding = {'No':0, 'Yes':1}
data['Card on Dark Web'] = data['Card on Dark Web'].map(dark_web_encoding)
unique_values = data['Card on Dark Web'].unique()
print(f'Card on Dark Web unique values: {unique_values}')
display(data)

In [ ]:
# Change 'Per Capita Income - Zipcode', 'Yearly Income - Person' and 'Total Debt' from string to float
data['Per Capita Income - Zipcode'] = data['Per Capita Income - Zipcode'].str.replace('$','').astype(float)
data['Yearly Income - Person'] = data['Yearly Income - Person'].str.replace('$','').astype(float)
data['Total Debt'] = data['Total Debt'].str.replace('$','').astype(float)
data

In [ ]:
# save csv
data_path = os.path.join(project_root, 'data/preprocessed_data_v001.csv')
data.to_csv(data_path)

In [ ]:
# save a subset of the data as a more light weight development data version
data_subset = (data
               .sample(1000000, random_state=42)
               .sort_values(by='Datetime')
               .reset_index(drop=True)
              )

data_path = os.path.join(project_root, 'data/preprocessed_data_small_v001.csv')
data_subset.to_csv(data_path)